# GPT-4o (Extraction 2.2)

In [1]:
from utils.image2base64 import image2base64
from utils.imageWithAxis2Base64 import imageWithAxis2Base64
from utils.getImageSize import getImageSize

from dotenv import load_dotenv, find_dotenv
import os

from langchain_openai import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage

load_dotenv(find_dotenv())

model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4o",
    model_kwargs={"top_p": 0, "seed": 42},
    temperature=0,
)

In [3]:
pz_path = "../data/processed/bpläne/F11-01-PZ-BB.png"
pze_path = "../data/processed/bpläne/F11-01-PZE-BB.png"

#base64_pz = imageWithAxis2Base64(pz_path)
base64_pz = image2base64(pz_path)
base64_pze = image2base64(pze_path)

print(getImageSize(pze_path))
print(getImageSize(pz_path))

[768, 1362]
[1038, 768]


In [11]:
msg1 = model.invoke([
    AIMessage(
        content=[
            {
            "type": "text",
            "text": "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren."
            }
        ]
    ),
    HumanMessage(
        content=[
        {
            "type": "text",
            "text": 'Die Nutzungsschablone in der vorliegenden Zeichenerklärung ist durch eine rote Bounding-Box markiert. Extrahiere die Struktur der Nutzungsschablone in ASCII-Tabellenform.'
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_pze}",
                "detail": "high"
            }
        },
        ]
    ),
])

print(msg1.content)

Hier ist die extrahierte Struktur der Nutzungsschablone in ASCII-Tabellenform:

```
+-------------------+-------------------+
| Art der Nutzung   | Gebäudeklasse     |
+-------------------+-------------------+
| Grundflächenzahl  | Geschossflächen-  |
| (GRZ)             | zahl (GFZ)        |
+-------------------+-------------------+
| Bauweise          |                   |
+-------------------+-------------------+
```


In [15]:
# CLEANUP -> FIX COMMON MISTAKES
import re

regex_pattern = r'Gebäudeklasse'
replacement_word = 'Gebäudehöhe'
msg1.content = re.sub(regex_pattern, replacement_word, msg1.content)
print(msg1.content)


Hier ist die extrahierte Struktur der Nutzungsschablone in ASCII-Tabellenform:

```
+-------------------+-------------------+
| Art der Nutzung   | Gebäudehöhe     |
+-------------------+-------------------+
| Grundflächenzahl  | Geschossflächen-  |
| (GRZ)             | zahl (GFZ)        |
+-------------------+-------------------+
| Bauweise          |                   |
+-------------------+-------------------+
```


In [16]:
msg2 = model.invoke([
    AIMessage(
        content=[
            {
            "type": "text",
            "text": "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren."
            },
            {
                "type": "text",
                "text": f"{msg1.content}"
            }
        ]
    ),
    HumanMessage(
        content=[
        {
            "type": "text",
            "text": 'Fasse alle relevanten Definitionen und Zeichen bezüglich der Schlagwörter aus der ASCII-Nutzungsschablone aus der Zeichenerklärung zusammen. Ausgabe im JSON-Format: {<Schlagwort>: {"Definition"="", "Zeichen": ""}}'
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_pze}",
                "detail": "high"
            }
        },
        ]
    ),
])

print(msg2.content)

Hier sind die relevanten Definitionen und Zeichen bezüglich der Schlagwörter aus der ASCII-Nutzungsschablone im JSON-Format:

```json
{
  "Art der Nutzung": {
    "Definition": "Nutzungsschablone",
    "Zeichen": "GE"
  },
  "Gebäudehöhe": {
    "Definition": "max. zulässige Gebäudehöhe über Erdgeschossfußbodenhöhe",
    "Zeichen": "GH=max.12,00"
  },
  "Grundflächenzahl (GRZ)": {
    "Definition": "Grundflächenzahl (§ 19 BauNVO)",
    "Zeichen": "0,6"
  },
  "Geschossflächenzahl (GFZ)": {
    "Definition": "Geschossflächenzahl (§ 20 BauNVO)",
    "Zeichen": "1,2"
  },
  "Bauweise": {
    "Definition": "Allgemeine Bauweise (§ 22 BauNVO)",
    "Zeichen": "-"
  }
}
```


In [17]:
msg3 = model.invoke([
    AIMessage(
        content=[
            {
            "type": "text",
            "text": "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren."
            },
            {
                "type": "text",
                "text": f"{msg1.content}"
            },
            {
                "type": "text",
                "text": f"{msg2.content}"
            }
        ]
    ),
    HumanMessage(
        content=[
        {
            "type": "text",
            #"text": 'Extrahiere die Informationen aus den rot umrandeten Nutzungsschablonen in der vorliegenden Planzeichnung. Ausgabe im JSON-Format.'
            "text": "Die Nutzungsschablonen in der vorliegenden Planzeichnung sind durch rote Bounding-Boxen markiert. Gebe die jeweiligen Werte der Nutzungsschablonen im JSON-Format aus."
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_pz}",
                "detail": "high"
            }
        },
        ]
    ),
])

print(msg3.content)

Hier sind die extrahierten Werte der Nutzungsschablonen im JSON-Format:

```json
[
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "GH=max.12,00",
    "Grundflächenzahl (GRZ)": "0,6",
    "Geschossflächenzahl (GFZ)": "1,2",
    "Bauweise": "-"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "GH=max.12,00",
    "Grundflächenzahl (GRZ)": "0,6",
    "Geschossflächenzahl (GFZ)": "1,2",
    "Bauweise": "-"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "GH=max.12,00",
    "Grundflächenzahl (GRZ)": "0,6",
    "Geschossflächenzahl (GFZ)": "1,2",
    "Bauweise": "-"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "GH=max.12,00",
    "Grundflächenzahl (GRZ)": "0,6",
    "Geschossflächenzahl (GFZ)": "1,2",
    "Bauweise": "-"
  }
]
```
